 **Exercise 2.04**

Fitting a Simple Linear Regression Model Using the Statsmodels 
formula API

In [ ]:
#importing modules
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import pandas as pd
import numpy as np
import patsy
from statsmodels.graphics.correlation import plot_corr
from sklearn.model_selection import train_test_split
plt.style.use('seaborn')



LOADING MIAMI_HOUSING DATASET AND DISPLAYING FIRST 5 ENTRIES

In [ ]:

miami_data = pd.read_csv\
('https://raw.githubusercontent.com/fenago/DSBook/main/miami-housing.csv')
miami_data.head()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


Check for missing values (null values) in the DataFram

In [ ]:
raw_data = miami_data.dropna()


In [ ]:
#Checking for duplicate data and removing it 
raw_data = raw_data.drop_duplicates()

In [ ]:
#List the column names of the DataFrame
list(raw_data.columns)

['LATITUDE',
 'LONGITUDE',
 'PARCELNO',
 'SALE_PRC',
 'LND_SQFOOT',
 'TOT_LVG_AREA',
 'SPEC_FEAT_VAL',
 'RAIL_DIST',
 'OCEAN_DIST',
 'WATER_DIST',
 'CNTR_DIST',
 'SUBCNTR_DI',
 'HWY_DIST',
 'age',
 'avno60plus',
 'month_sold',
 'structure_quality']

Renaming the DataFrame columns from 1 to 5, others are kept same

In [ ]:
renamedMiamiData = raw_data.rename\
(columns = {\
'LATITUDE':'LATITUDE',\
' LONGITUDE ':'lONGITUDE',\
'PARCELNO ':'Parcel Number',\
'SALE_PRC':'SalePrice',\
'LND_SQFOOT':'LandLength'})
renamedMiamiData.head()

,LATITUDE,LONGITUDE,PARCELNO,SalePrice,LandLength,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


In [ ]:
renamedMiamiData.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
LATITUDE,13932.0,2.572881e+01,1.406333e-01,2.543433e+01,2.562006e+01,2.573181e+01,2.585227e+01,2.597438e+01
LONGITUDE,13932.0,-8.032748e+01,8.919907e-02,-8.054217e+01,-8.040328e+01,-8.033891e+01,-8.025802e+01,-8.011975e+01
PARCELNO,13932.0,2.356496e+12,1.199290e+12,1.020008e+11,1.079160e+12,3.040300e+12,3.060170e+12,3.660170e+12
SalePrice,13932.0,3.999419e+05,3.172147e+05,7.200000e+04,2.350000e+05,3.100000e+05,4.280000e+05,2.650000e+06
LandLength,13932.0,8.620880e+03,6.070089e+03,1.248000e+03,5.400000e+03,7.500000e+03,9.126250e+03,5.706400e+04
TOT_LVG_AREA,13932.0,2.058045e+03,8.135385e+02,8.540000e+02,1.470000e+03,1.877500e+03,2.471000e+03,6.287000e+03
SPEC_FEAT_VAL,13932.0,9.562493e+03,1.389097e+04,0.000000e+00,8.100000e+02,2.765500e+03,1.235225e+04,1.750200e+05
RAIL_DIST,13932.0,8.348549e+03,6.178027e+03,1.050000e+01,3.299450e+03,7.106300e+03,1.210260e+04,2.962150e+04
OCEAN_DIST,13932.0,3.169099e+04,1.759508e+04,2.361000e+02,1.807935e+04,2.854175e+04,4.431065e+04,7.574490e+04
WATER_DIST,13932.0,1.196029e+04,1.193299e+04,0.000000e+00,2.675850e+03,6.922600e+03,1.920000e+04,5.039980e+04


Divide the DataFrame into training and test sets

In [ ]:
X = renamedMiamiData.drop('SalePrice', axis = 1)
y = renamedMiamiData[['SalePrice']]
seed = 10
test_data_size = 0.3
X_train, X_test, \
y_train, y_test = train_test_split(X, y, \
test_size = test_data_size, \
random_state = seed)
train_data = pd.concat([X_train, y_train], axis = 1)
test_data = pd.concat([X_test, y_test], axis = 1)

 Defining a linear regression model and storing it in a variable

In [ ]:
#----Write your code here----#



#-------Solution:---------#
#Regression_model = smf.ols\
#              (formula='SalePrice ~ LandLength',\
#               data=train_data)

Calling the .fit method of the model instance and assigning the results of the method to a linearModelResult variable,

In [ ]:
FinalLinearModel = Regression_model.fit()

Printing  the summary stored in Model -  Regression analysis results

In [ ]:
print(FinalLinearModel.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     1587.
Date:                Fri, 04 Mar 2022   Prob (F-statistic):          9.19e-322
Time:                        13:42:35   Log-Likelihood:            -1.3679e+05
No. Observations:                9752   AIC:                         2.736e+05
Df Residuals:                    9750   BIC:                         2.736e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.336e+05   5198.776     44.931      0.0